## EDA

Import packages

In [2]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

Load Data

In [3]:
df = pd.read_csv('input/accepted_2007_to_2018Q4.csv.gz', compression='gzip', low_memory=True)

## METADATA

In [ ]:
def BuildMetadata(dataframe,target,id): 

    train = dataframe.rename(columns={target:'target',id:'id'})
    # Verifica os tipos de variáveis presentes na tabela de treino
    t = []
    for i in train.columns:
            t.append(train[i].dtype)

    n = []
    for i in train.columns:
            n.append(i)

    aux_t = pd.DataFrame(data=t,columns=["Tipos"])
    aux_n = pd.DataFrame(data=n,columns=["Features"])
    # df_tipovars = pd.concat([aux_n, aux_t], axis=1, join_axes=[aux_n.index])
    df_tipovars = pd.concat([aux_n, aux_t.reindex(aux_n.index)], axis=1)


    data = []
    for f in train.columns:

        # Definindo o papel das variáveis:
        if f == 'target':
            role = 'target'
        elif f == 'id':
            role = 'id'
        else:
            role = 'input'

        # Definindo o tipo das variáveis: nominal, ordinal, binary ou interval
        if f == 'target':
            level = 'binary'
        if train[f].dtype == 'object' or f == 'id': 
            level = 'nominal'
        elif train[f].dtype in ['float','float64'] :
            level = 'interval'
        elif train[f].dtype in ['int','int64'] :
            level = 'ordinal'

        # Todas variáveis são incializadas com keep exceto o id
        keep = True
        if f == 'id':
            keep = False

        # Definindo o tipo das variáveis da tabela de entrada
        dtype = train[f].dtype

        # Criando a lista com todo metadados
        f_dict = {
            'Features': f,
            'Role': role,
            'Level': level,
            'Keep': keep,
            'Tipo': dtype
        }
        data.append(f_dict)

    meta = pd.DataFrame(data, columns=['Features', 'Role', 'Level', 'Keep', 'Tipo'])

    # Quantidade de domínios distintos para cada cariável do tipo ordinal e nominal
    card = []

    v = train.columns
    for f in v:
        dist_values = train[f].value_counts().shape[0]
        f_dict = {
                'Features': f,
                'Cardinality': dist_values
            }
        card.append(f_dict)

    card = pd.DataFrame(card, columns=['Features', 'Cardinality'])

    metadados_train = pd.merge(meta, card, on='Features')

    return metadados_train 

In [ ]:
id = '' 
target = ''

features = [
'platform', 
'graphics_memory_size', 
'system_memory_size',
'adcquision_network', 
]

# 
select_cols = [id] + features + [target]
select_cols

In [ ]:
metadata = BuildMetadata(df[select_cols],target,id)

In [ ]:
metadata.to_csv('artifacts/metadata.csv', sep=',', encoding='utf-8',index=False)
df[select_cols].to_csv('input/df.csv', sep=',', encoding='utf-8',index=False)